## Download video from github

In [1]:
!wget https://github.com/AhmedIbrahimai/Vehicle-detection-and-counting-using-openCV/blob/main/video.mp4?raw=true
#dowbload video from enj ahmed ibrahim github

--2025-01-28 14:58:11--  https://github.com/AhmedIbrahimai/Vehicle-detection-and-counting-using-openCV/blob/main/video.mp4?raw=true
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/AhmedIbrahimai/Vehicle-detection-and-counting-using-openCV/raw/refs/heads/main/video.mp4 [following]
--2025-01-28 14:58:12--  https://github.com/AhmedIbrahimai/Vehicle-detection-and-counting-using-openCV/raw/refs/heads/main/video.mp4
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AhmedIbrahimai/Vehicle-detection-and-counting-using-openCV/refs/heads/main/video.mp4 [following]
--2025-01-28 14:58:12--  https://raw.githubusercontent.com/AhmedIbrahimai/Vehicle-detection-and-counting-using-openCV/refs/heads/main/video.mp4
Resolving raw.githubusercontent.com (raw.githubuserco

### convertfrom  mp4  to avi

In [2]:
!ffmpeg -i /content/video.mp4?raw=true video.mp4?raw=true.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### display video

In [ ]:

from IPython.display import HTML
from base64 import b64encode

link = '/content/video.mp4?raw=true.mp4'

mp4 = open(link,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{data_url}"></video>')

# Main Code
### & Algorithm

In [3]:
import cv2
import numpy as np
from time import sleep
import time

# Record the start time
start_time = time.time()

# Constants
LARGURA_MIN = 80
ALTURA_MIN = 80
OFFSET = 6
POS_LINHA = 550
DELAY = 60
WIDTH, HEIGHT = 370, 370

# Global variables
detec = []
carros = 0

# Helper function to calculate the center of a rectangle
def pega_centro(x, y, w, h):
    cx = x + w // 2
    cy = y + h // 2
    return cx, cy

# Alternative method to calculate the center using moments
def pega_centro_moments(contour):
    M = cv2.moments(contour)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        return cx, cy
    return None, None

# Initialize video capture and background subtractor
cap = cv2.VideoCapture('/content/video.mp4?raw=true')
subtracao = cv2.bgsegm.createBackgroundSubtractorMOG()


WIDTH = int(cap.get(3))
HEIGHT=int(cap.get(4))
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (WIDTH, HEIGHT))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    sleep(1 / DELAY)

    # Preprocessing
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)
    img_sub = subtracao.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contornos, _ = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw detection line
    cv2.line(frame, (25, POS_LINHA), (1200, POS_LINHA), (176, 130, 39), 2)

    for contour in contornos:
        x, y, w, h = cv2.boundingRect(contour)

        # Validate contour dimensions
        if w >= LARGURA_MIN and h >= ALTURA_MIN:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            centro = pega_centro(x, y, w, h)
            detec.append(centro)
            cv2.circle(frame, centro, 4, (0, 0, 255), -1)

    # Check if any object crossed the detection line
    for (cx, cy) in detec:
        if POS_LINHA - OFFSET < cy < POS_LINHA + OFFSET:
            carros += 1
            cv2.line(frame, (25, POS_LINHA), (1200, POS_LINHA), (0, 127, 255), 3)
            detec.remove((cx, cy))
            print(f"No. of cars detected: {carros}")



    # Display vehicle count
    cv2.putText(frame, f"VEHICLE COUNT: {carros}", (320, 70), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)

    # Write frame to output video
    out.write(frame)

    # just for test on 25 seconds of the video
    if time.time() - start_time > 50:
        print("50 seconds have passed. Breaking out of the loop.")
        break

# Clean up
cap.release()
out.release()


No. of cars detected: 1
No. of cars detected: 2
No. of cars detected: 3
No. of cars detected: 4
No. of cars detected: 5
No. of cars detected: 6
No. of cars detected: 7
No. of cars detected: 8
No. of cars detected: 9
No. of cars detected: 10
No. of cars detected: 11
No. of cars detected: 12
No. of cars detected: 13
No. of cars detected: 14
No. of cars detected: 15
No. of cars detected: 16
No. of cars detected: 17
No. of cars detected: 18
No. of cars detected: 19
No. of cars detected: 20
No. of cars detected: 21
No. of cars detected: 22
50 seconds have passed. Breaking out of the loop.


## convert the Extention and display the video

In [4]:
!ffmpeg -i /content/output.avi output.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [5]:

from IPython.display import HTML
from base64 import b64encode

link = '/content/output.mp4'


mp4 = open(link,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{data_url}"></video>')